In [1]:
import os
import sys
current = os.path.dirname(os.path.realpath("match.ipynb"))
parent = os.path.dirname(current)
sys.path.append(parent)
from transformers import CLIPProcessor, CLIPModel
import torch
import torchvision
from torchvision.models import resnet50
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import clip
from PIL import Image
import requests
import torch.hub
import time
import pickle
import math

from match_utils import matching, stats, proggan, nethook, dataset, models

/home/amil/anaconda3/envs/rosetta/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-14 20:53:37.849249: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-14 20:53:37.890027: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 20:53:38.522624: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could n

In [2]:
device = torch.device('cuda:0')

In [3]:
gan, gan_layers = models.load_gan("biggan", device)
discr, discr_layers = models.load_discr("dino", device)

Using cache found in /home/amil/.cache/torch/hub/facebookresearch_dino_main
/home/amil/anaconda3/envs/rosetta/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/amil/anaconda3/envs/rosetta/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


### Run Matching

In [ ]:
batch_size = 16
epochs = 100
classidx = 207
save_path = "/home/amil/Rosetta/matches"
z_dataset, c_dataset = dataset.create_dataset(gan, "biggan", batch_size, epochs, classidx, device)

matching.activ_match_gan(gan = gan, gan_layers = gan_layers,
                        discr = discr, discr_layers = discr_layers, 
                        gan_mode = "biggan", 
                        discr_mode = "dino", 
                        dataset = (z_dataset, c_dataset),
                        epochs = epochs,
                        batch_size = batch_size,
                        save_path = save_path,
                        device = device)
end = time.time()

100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 14.07it/s]


Finished Iterating for Stats
Done
Starting Activation Matching


 53%|███████████████████████████████████████████▍                                      | 53/100 [02:22<02:06,  2.69s/it]